In [ ]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

In [ ]:
df_res = pd.read_csv('df_res.csv')

wind_array = df_res.to_numpy()

In [ ]:
# window lenght is set 500
window = 500
step = int(window/2)
# 5 = window/step
length = 1 + (len(wind_array) // window - 1)*2
nrm = [[] for j in range(length)]

for i in range(0, length):
    nrm[i] = wind_array[i*(step):i*(step) + window]

In [ ]:
np.shape(nrm)

(23, 4000, 69)

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform, RandomNormal
from matplotlib import pyplot as plt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from tqdm import tqdm

tfd = tfp.distributions

### 2. WCDF

profile modeling

In [ ]:
from scipy.optimize import curve_fit

def f(x, a, b, c):
  # return 1 - np.exp(-(x/c)**k)
  return a*(x**3) + b*(x**2) + c*x

def find_nearest(a, a0):
    "Element in nd array `a` closest to the scalar value `a0`"
    idx = np.abs(a - a0).argmin()
    return idx

In [ ]:
np.shape(nrm)

(23, 4000, 69)

In [ ]:
for i in tqdm(range(23)):
  nrm0 = nrm[i]
  speed0 = nrm0[:, 0::3]
  power0 = nrm0[:, 1::3]
  label0 = nrm0[:, 2::3]

  speed = nrm0[:, 0::3]
  power = nrm0[:, 1::3]
  omega = 0.2
  std = np.std(speed0[:,0])
  speed_new = [[] for _ in range(23)]
  power_new = [[] for _ in range(23)]
  speed_new[0] = speed[:,0]
  power_new[0] = power[:,0]

  for j in range(1, 23):
    for t in range(len(speed[:,0])):
      idx = find_nearest(speed[:,j], speed[:,0][t])
      tmp = speed[:,j].flat[idx]
      if abs(tmp - speed[:,0][t]) <  omega*std:
        speed_new[j].append(tmp)
        power_new[j].append(power[:,j].flat[idx])

  model_params = []
  for k in range(23):
    print("Current segment: %s; WT: %s"%(i, k))
    speed_list = np.array(speed_new[k])
    power_list = np.array(power_new[k])
    power_list = (power_list - 0)/4000 # 设置在0-1之间
    power_list[power_list>1] = 1
    power_list[power_list<0] = 0


    # save model weights
    speed_fit = []
    power_fit = []
    for n in range(len(speed_list)):
      if speed_list[n] >=4 and speed_list[n] <=12:
        speed_fit.append(speed_list[n])
        power_fit.append(power_list[n])

    popt, pcov = curve_fit(f, speed_fit, power_fit, maxfev=80000)
    model_params.append(popt)

  np.save('baseline/WCDF_4000/%s.npy'%i, model_params)

model result plot

In [ ]:
i = 0
nrm0 = nrm[i]
speed0 = nrm0[:, 0::3]
power0 = nrm0[:, 1::3]
label0 = nrm0[:, 2::3]

speed = nrm0[:, 0::3]
power = nrm0[:, 1::3]
omega = 0.2
std = np.std(speed0[:,0])
speed_new = [[] for _ in range(23)]
power_new = [[] for _ in range(23)]
speed_new[0] = speed[:,0]
power_new[0] = power[:,0]

for j in range(1, 23):
  for t in range(len(speed[:,0])):
    idx = find_nearest(speed[:,j], speed[:,0][t])
    tmp = speed[:,j].flat[idx]
    if abs(tmp - speed[:,0][t]) <  omega*std:
      speed_new[j].append(tmp)
      power_new[j].append(power[:,j].flat[idx])

weight_list = np.load('baseline/WCDF_4000/%s.npy'%i)

for k in range(23):
  print("Current segment: %s; WT: %s"%(i, k))
  speed_list = np.array(speed_new[k])
  power_list = np.array(power_new[k])
  power_list = (power_list - 0)/4000 # set between 0 and 1
  power_list[power_list>1] = 1
  power_list[power_list<0] = 0


  # save model weights
  speed_fit = []
  power_fit = []
  for n in range(len(speed_list)):
    if speed_list[n] >=4 and speed_list[n] <=12:
      speed_fit.append(speed_list[n])
      power_fit.append(power_list[n])

  plt.figure(figsize=(10, 5))

  pred = f(np.array(speed_fit), weight_list[k][0], weight_list[k][1], weight_list[k][2])
  plt.scatter(speed_fit, power_fit, c='blue')
  plt.scatter(speed_fit, pred, c='r')

performance degradation monitoring

In [ ]:
from scipy.stats import f

predict_matrix = np.zeros((23, 23))

for i in range(23):
  weight_list = np.load('baseline/WCDF_4000/%s.npy'%i)

  S = np.cov(weight_list.T)
  S_inv = np.linalg.inv(S)

  X_bar = np.mean(weight_list, axis=0)
  T2 = [np.array((np.matrix(weight_list[i]-X_bar)*np.matrix(S_inv)*np.matrix(weight_list[i] - X_bar).T))[0][0] for i in range(len(weight_list))]

  p = 3
  m = len(weight_list)
  alpha=0.05
  a, b=f.interval(1-alpha, dfn=p, dfd=m-p)

  UCL = (p*(m+1)*(m-1)*b)/(m**2-m*p)

  for k in range(23):
    if T2[k] > UCL:
      predict_matrix[i][k] = 1

In [ ]:
np.shape(nrm)

(197, 500, 69)

In [ ]:
degree_matrix = np.zeros((23, 23))

length = 4000

for i in tqdm(range(23)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 23/23 [00:00<00:00, 10894.30it/s]


In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.16883116883116883
precision 0.9285714285714286
f1-score 0.28571428571428575


GR + performance comparison

In [ ]:
from sklearn.gaussian_process.kernels import RBF, DotProduct, WhiteKernel, ConstantKernel as C
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.preprocessing import StandardScaler
import joblib

In [ ]:
for i in tqdm(range(23)): # len(nrm)
  nrm0 = nrm[i]
  speed0 = nrm0[:, 0::3]
  power0 = nrm0[:, 1::3]
  label0 = nrm0[:, 2::3]

  speed = nrm0[:, 0::3]
  power = nrm0[:, 1::3]
  omega = 0.2
  std = np.std(speed0[:,0])
  speed_new = [[] for _ in range(23)]
  power_new = [[] for _ in range(23)]
  speed_new[0] = speed[:,0]
  power_new[0] = power[:,0]

  for j in range(1, 23):
    for t in range(len(speed[:,0])):
      idx = find_nearest(speed[:,j], speed[:,0][t])
      tmp = speed[:,j].flat[idx]
      if abs(tmp - speed[:,0][t]) <  omega*std:
        speed_new[j].append(tmp)
        power_new[j].append(power[:,j].flat[idx])

  model_params = []
  for k in range(23):
    print("Current segment: %s; WT: %s"%(i, k))
    speed_list = np.array(speed_new[k])
    power_list = np.array(power_new[k])
    power_list = (power_list - 0)/4000 # set between 0 and 1
    power_list[power_list>1] = 1
    power_list[power_list<0] = 0

    # save model weights
    speed_fit = []
    power_fit = []
    for n in range(len(speed_list)):
      if speed_list[n] >=4 and speed_list[n] <=12:
        speed_fit.append(speed_list[n])
        power_fit.append(power_list[n])

    kernel = RBF(10, (0.5, 2))
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9,alpha=1)
    gp.fit(np.array(speed_fit)[::2].reshape(-1, 1), np.array(power_fit)[::2])
    joblib.dump(gp, 'baseline/gp_4000/%s_%s.pkl'%(i, k))

In [ ]:
power_fit

In [ ]:
model = joblib.load('baseline/gp/0_0.pkl')

In [ ]:
res, res_sigmas = model.predict(np.array(speed_fit).reshape(-1, 1), return_std=True)

In [ ]:
preds, sigmas = gp.predict(np.array(speed_fit).reshape(-1, 1), return_std=True)

In [ ]:
plt.scatter(speed_fit, res)

Distance matrix calculation

In [ ]:
def difference_calculation(res1, res2):
  power_bar = np.linspace(0, 1, 51)
  interval = 0.02

  res1_df = pd.Series(res1)
  s1 = pd.cut(res1_df, power_bar).value_counts(sort=False).values

  mu_array = np.zeros(50)
  delta_array = np.zeros(50)
  for k in range(len(res1)):
    ind = np.max(np.where((res1[k] - power_bar)>=0))
    mu_array[ind] += res1[k]
    delta_array[ind] += abs(res2[k] - res1[k])

  for n in range(len(mu_array)):
    if mu_array[n] > 0:
      mu_array[n] = mu_array[n]/s1[n]
      delta_array[n] =  delta_array[n]

  pai_s1 = s1/(len(s1))
  power_difference = np.sum(delta_array*pai_s1)
  mu_array = mu_array + 0.01
  scaled_power_difference = power_difference/np.sum(mu_array*pai_s1) 
  return scaled_power_difference

In [ ]:
distance_matrix = np.zeros((23, 23))

for i in tqdm(range(23)):
  nrm0 = nrm[i]

  speed = nrm0[:, 0::3]
  power = nrm0[:, 1::3]

  max_speed = min(np.max(speed.flatten()), 12)
  min_speed = max(np.min(speed.flatten()), 4)

  speed_list = np.linspace(min_speed, max_speed, 100)

  for j in range(23):
    distance_all = []
    model1 = joblib.load('baseline/gp_4000/%s_%s.pkl'%(i,j))
    res1 = model1.predict(speed_list.reshape(-1, 1), return_std=False)
    res1[res1<0] = 0
    for k in range(23):
      if k == j:
        continue

      model2 = joblib.load('baseline/gp_4000/%s_%s.pkl'%(i,k))
      res2 = model2.predict(speed_list.reshape(-1, 1), return_std=False)
      res2[res2<0] = 0

      # difference calculation
      difference = difference_calculation(res1, res2)
      distance_all.append(difference)

    sort_array = np.argsort(distance_all)[:-3]
    filter_distance = [distance_all[i] for i in sort_array]
    distance_matrix[i][j] = np.mean(filter_distance)

100%|██████████| 23/23 [15:51<00:00, 41.39s/it]


In [ ]:
np.save('baseline/gp_4000/distance_matrix.npy', distance_matrix)

Distance matrix monitoring

In [ ]:
distance_matrix = np.load('baseline/gp_500/distance_matrix.npy')

distance_matrix[np.isnan(distance_matrix)] = 50

In [ ]:
np.shape(distance_matrix)

(46, 23)

In [ ]:
def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [ ]:
from tqdm import tqdm
import scipy

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(46)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.5)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

100%|██████████| 46/46 [02:45<00:00,  3.60s/it]


In [ ]:
predict_matrix = np.zeros((46, 23))

for i in range(46):
  for j in range(23):
    if distance_matrix[i][j] > UCL_list[i]:
      predict_matrix[i][j] = 1
    else:
      pass

In [ ]:
degree_matrix = np.zeros((46, 23))

length = 500

for i in tqdm(range(46)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/length # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

100%|██████████| 46/46 [00:00<00:00, 19786.48it/s]


In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass

precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 1.0
precision 0.28888888888888886
f1-score 0.4482758620689655


Data stream

In [ ]:
from scipy.stats import pearsonr,spearmanr

In [ ]:
model_params = [[] for _ in range(3)]
for i in range(47):
  weight_list = np.load('baseline/WCDF/%s.npy'%i)
  model_params[0].append(weight_list[0][0])
  model_params[1].append(weight_list[0][1])
  model_params[2].append(weight_list[0][2])

In [ ]:
np.shape(model_params)

(3, 47)

In [ ]:
print(pearsonr(model_params[0], model_params[1])[0])
print(pearsonr(model_params[0], model_params[2])[0])
print(pearsonr(model_params[1], model_params[2])[0])

-0.9845351427211695
0.9654242463587768
-0.9947289304792464


In [ ]:
from scipy.stats import shapiro

shapiro(model_params[2], )

ShapiroResult(statistic=0.9741247296333313, pvalue=0.3767985999584198)